In [1]:
import pandas as pd
import numpy as np
import nltk
import openai

### 1.Azure Embeddings

In [2]:
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = "2023-03-15-preview"
openai.api_key = ""

def get_embedding_azure(text, embedding_model, deployment_id):
    response = openai.Embedding.create(
        input=text,
        engine=embedding_model,
        deployment_id = deployment_id
    )
    return response['data'][0]['embedding']


def compute_embeddings(df, asr_column_name: str) :
    # Compute Azure Embeddings
    df[f"embedding_{asr_column_name}"] = df[asr_column_name].apply(lambda x: get_embedding_azure(x,embedding_model = "", deployment_id = ""))
    return df

In [4]:
df = pd.read_csv("../1.Database/dataset_out_of_place_open_responses.csv")

#### 1.1 Preprocess text

In [ ]:
from clean_asr_service import CleanASRService

cleanASRservice = CleanASRService()

# Normalize text, delete duplicates etc
df["cleaned_asr"] = df.asr.apply(lambda x: cleanASRservice.execute(x, delete_stopwords=False))

# Normalize text, delete duplicates etc + delete stopwords
# Not use "no" and "sí" as stopwords
stopword_es = nltk.corpus.stopwords.words('spanish')
stopword_es.remove("sí")
stopword_es.remove("no")
df["cleaned_asr_without_stopw"] = df.asr.apply(lambda x: cleanASRservice.execute(x, stopwords_list=stopword_es,delete_stopwords=True))

#### 1.2 Compute Azure Embeddings

In [6]:
df = compute_embeddings(df,"asr")
df = compute_embeddings(df,"cleaned_asr")
df = compute_embeddings(df,"cleaned_asr_without_stopw")
# Save encoding database
df.to_csv(f"Azure_{embedding_model}}_embeddings.csv")